In [50]:
from utilities.utilities import load_data, get_records_by_region, create_column, finalize_dataframe, save_string_to_file, get_extreme_values, create_directory_structure, save_table, save_report
# settings
region_column_name = 'Region'

category = 'security_layer'
column_name_to_results_global = 'Global #'
create_directory_structure()



source_df = load_data('security_layer_checker')

In [51]:
# sanity dataset

In [52]:
# Analyze of key length by region

# settings
column_to_sort = 'Without SSL (Public) %'
sort_ascending = False
table_name = 'key_length_by_region'

columns_to_display = [region_column_name.title(), column_name_to_results_global]
analysis_df = get_records_by_region(source_df)

# create columns
# Column creation with distribution of records without SSL by region
only_public = 'category == "Public"'
only_private = 'category == "Private"'
create_column(source_df=source_df, analysis_dataframe=analysis_df, column_name='Without SSL (Public)', criteria=f'grade == "M" & {only_public}', columns_to_display=columns_to_display)
create_column(source_df=source_df, analysis_dataframe=analysis_df, column_name='Without SSL (Private)', criteria=f'grade == "M" & {only_private}', columns_to_display=columns_to_display)
# Creating column with the distribution of Key Length by region
key_lengths = [256, 384, 1024, 2048, 3072, 4096]
for key_length in key_lengths:
    algorithm = 'RSA' if key_length >= 1024 else 'ECC'
    create_column(source_df=source_df, analysis_dataframe=analysis_df, column_name=f'{key_length} ({algorithm}) (Public)', criteria=f'grade != "M" & key_size == {key_length} & {only_public}', columns_to_display=columns_to_display)
    create_column(source_df=source_df, analysis_dataframe=analysis_df, column_name=f'{key_length} ({algorithm}) (Private)', criteria=f'grade != "M" & key_size == {key_length} & {only_private}', columns_to_display=columns_to_display)


# Finalize dataframe
analysis_df = finalize_dataframe(dataframe=analysis_df, column_to_sort=column_to_sort, ascending=sort_ascending, columns_to_display=columns_to_display)
display(analysis_df)

# save to csv
save_table(analysis_df, category=category, table_name=table_name)

,Region,Global #,Without Ssl (Public) #,Without Ssl (Public) %,Without Ssl (Private) #,Without Ssl (Private) %,256 (Ecc) (Public) #,256 (Ecc) (Public) %,256 (Ecc) (Private) #,256 (Ecc) (Private) %,...,2048 (Rsa) (Private) #,2048 (Rsa) (Private) %,3072 (Rsa) (Public) #,3072 (Rsa) (Public) %,3072 (Rsa) (Private) #,3072 (Rsa) (Private) %,4096 (Rsa) (Public) #,4096 (Rsa) (Public) %,4096 (Rsa) (Private) #,4096 (Rsa) (Private) %
0,California,2,0,0.0,0,0.0,0,0.0,0,0.0,...,2,100.0,0,0.0,0,0.0,0,0.0,0,0.0
1,Colorado,1,0,0.0,0,0.0,0,0.0,0,0.0,...,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0
2,Florida,1,0,0.0,0,0.0,0,0.0,0,0.0,...,1,100.0,0,0.0,0,0.0,0,0.0,0,0.0
3,Georgia,1,0,0.0,0,0.0,0,0.0,0,0.0,...,1,100.0,0,0.0,0,0.0,0,0.0,0,0.0
4,Michigan,1,0,0.0,0,0.0,0,0.0,0,0.0,...,1,100.0,0,0.0,0,0.0,0,0.0,0,0.0
5,Minnesota,1,0,0.0,0,0.0,0,0.0,0,0.0,...,1,100.0,0,0.0,0,0.0,0,0.0,0,0.0
6,New York,1,0,0.0,0,0.0,0,0.0,0,0.0,...,0,0.0,0,0.0,0,0.0,0,0.0,1,100.0
7,Texas,2,0,0.0,0,0.0,0,0.0,0,0.0,...,2,100.0,0,0.0,0,0.0,0,0.0,0,0.0
8,Total,10,0,0.0,0,0.0,0,0.0,0,0.0,...,8,80.0,0,0.0,0,0.0,0,0.0,1,10.0


In [53]:
# Report in latex
report_results = get_extreme_values(analysis_df)
report_name = 'key_length_by_region'
hei_totals_public = []
hei_totals_private = []
for key_length in key_lengths:
    algorithm = 'RSA' if key_length >= 1024 else 'ECC'
    hei_totals_public.append(format(report_results.get("Total").get(f'{key_length} ({algorithm.title()}) (Public) %'), ".2f"))
    hei_totals_private.append(format(report_results.get("Total").get(f'{key_length} ({algorithm.title()}) (Private) %'), ".2f"))


report_figure = f"""
\\begin{{figure}}[htbp]
    \centering
    \includegraphics[width=0.48\\textwidth]{{charts/{report_name}.pdf}}
    \caption{{Distribution of the length of \gls{{ssl}}/\gls{{tls}} digital certificate signing keys.}}\label{{fig:key-length}}
\end{{figure}}
"""

report = f'{report_figure}\n\n'
report += f"""
Fig.~\\ref{{fig:key-length}} presents an overview of the length of \gls{{ssl}}/\gls{{tls}} digital certificate signing keys at \glspl{{hei}} in \countryName.

According to the data, in analyzed public institutions, """

for i in range(len(key_lengths)):
    report += f'{hei_totals_public[i]}\% use {key_lengths[i]} bits keys, '

report += f"""and in analyzed private institutions, """

for i in range(len(key_lengths)):
    report += f'{hei_totals_private[i]}\% use {key_lengths[i]} bits keys, '

report += f'respectively. In terms of regional differences:'
for key_length in key_lengths:
    algorithm = 'RSA' if key_length >= 1024 else 'ECC'
    report += f"""
private institutions in {report_results.get(f"{key_length} ({algorithm.title()}) (Private) %").get("top_regions")[0][0]} ({format(report_results.get(f"{key_length} ({algorithm.title()}) (Private) %").get("top_regions")[0][1], ".2f")}\%), {report_results.get(f"{key_length} ({algorithm.title()}) (Private) %").get("top_regions")[1][0]} ({format(report_results.get(f"{key_length} ({algorithm.title()}) (Private) %").get("top_regions")[1][1], ".2f")}\%), and {report_results.get(f"{key_length} ({algorithm.title()}) (Private) %").get("top_regions")[2][0]} ({format(report_results.get(f"{key_length} ({algorithm.title()}) (Private) %").get("top_regions")[2][1], ".2f")}\%) use {key_length} bits keys the most often, while in {report_results.get(f"{key_length} ({algorithm.title()}) (Private) %").get("bottom_regions")[0][0]} ({format(report_results.get(f"{key_length} ({algorithm.title()}) (Private) %").get("bottom_regions")[0][1], ".2f")}\%), {report_results.get(f"{key_length} ({algorithm.title()}) (Private) %").get("bottom_regions")[1][0]} ({format(report_results.get(f"{key_length} ({algorithm.title()}) (Private) %").get("bottom_regions")[1][1], ".2f")}\%), and {report_results.get(f"{key_length} ({algorithm.title()}) (Private) %").get("bottom_regions")[2][0]} ({format(report_results.get(f"{key_length} ({algorithm.title()}) (Private) %").get("bottom_regions")[2][1], ".2f")}\%) use {key_length} bits keys the least often.

In terms of public institutions, {report_results.get(f"{key_length} ({algorithm.title()}) (Public) %").get("top_regions")[0][0]} ({format(report_results.get(f"{key_length} ({algorithm.title()}) (Public) %").get("top_regions")[0][1], ".2f")}\%), {report_results.get(f"{key_length} ({algorithm.title()}) (Public) %").get("top_regions")[1][0]} ({format(report_results.get(f"{key_length} ({algorithm.title()}) (Public) %").get("top_regions")[1][1], ".2f")}\%), and {report_results.get(f"{key_length} ({algorithm.title()}) (Public) %").get("top_regions")[2][0]} ({format(report_results.get(f"{key_length} ({algorithm.title()}) (Public) %").get("top_regions")[2][1], ".2f")}\%) use {key_length} bits keys the most often, while in {report_results.get(f"{key_length} ({algorithm.title()}) (Public) %").get("bottom_regions")[0][0]} ({format(report_results.get(f"{key_length} ({algorithm.title()}) (Public) %").get("bottom_regions")[0][1], ".2f")}\%), {report_results.get(f"{key_length} ({algorithm.title()}) (Public) %").get("bottom_regions")[1][0]} ({format(report_results.get(f"{key_length} ({algorithm.title()}) (Public) %").get("bottom_regions")[1][1], ".2f")}\%), and {report_results.get(f"{key_length} ({algorithm.title()}) (Public) %").get("bottom_regions")[2][0]} ({format(report_results.get(f"{key_length} ({algorithm.title()}) (Public) %").get("bottom_regions")[2][1], ".2f")}\%) use {key_length} bits keys the least often.
"""

print(report)
# save report to file txt
save_report(report=report, category=category, report_name=report_name)


\begin{figure}[htbp]
    \centering
    \includegraphics[width=0.48\textwidth]{charts/key_length_by_region.pdf}
    \caption{Distribution of the length of \gls{ssl}/\gls{tls} digital certificate signing keys.}\label{fig:key-length}
\end{figure}



Fig.~\ref{fig:key-length} presents an overview of the length of \gls{ssl}/\gls{tls} digital certificate signing keys at \glspl{hei} in \countryName.

According to the data, in analyzed public institutions, 0.00\% use 256 bits keys, 0.00\% use 384 bits keys, 0.00\% use 1024 bits keys, 10.00\% use 2048 bits keys, 0.00\% use 3072 bits keys, 0.00\% use 4096 bits keys, and in analyzed private institutions, 0.00\% use 256 bits keys, 0.00\% use 384 bits keys, 0.00\% use 1024 bits keys, 80.00\% use 2048 bits keys, 0.00\% use 3072 bits keys, 10.00\% use 4096 bits keys, respectively. In terms of regional differences:
private institutions in California (0.00\%), Colorado (0.00\%), and Florida (0.00\%) use 256 bits keys the most often, while in Californ

In [54]:
# Analyze of SSL Algorithms by region

# settings
column_to_sort = 'Without SSL (Public) %'
sort_ascending = False
table_name = 'SSL_Algorithms_by_region'

columns_to_display = [region_column_name.title(), column_name_to_results_global]
analysis_df = get_records_by_region(source_df)

# create columns
# Column creation with distribution of records without SSL by region
only_public = 'category == "Public"'
only_private = 'category == "Private"'
create_column(source_df=source_df, analysis_dataframe=analysis_df, column_name='Without SSL (Public)', criteria=f'grade == "M" & {only_public}', columns_to_display=columns_to_display)
create_column(source_df=source_df, analysis_dataframe=analysis_df, column_name='Without SSL (Private)', criteria=f'grade == "M" & {only_private}', columns_to_display=columns_to_display)
# Creating column with the distribution of SSL Algorithms by region
create_column(source_df=source_df, analysis_dataframe=analysis_df, column_name='RSA (Public)', criteria=f'grade != "M" & key_alg == "RSA" & {only_public}', columns_to_display=columns_to_display)
create_column(source_df=source_df, analysis_dataframe=analysis_df, column_name='RSA (Private)', criteria=f'grade != "M" & key_alg == "RSA" & {only_private}', columns_to_display=columns_to_display)
create_column(source_df=source_df, analysis_dataframe=analysis_df, column_name='ECC (Public)', criteria=f'grade != "M" & key_alg == "ECC" & {only_public}', columns_to_display=columns_to_display)
create_column(source_df=source_df, analysis_dataframe=analysis_df, column_name='ECC (Private)', criteria=f'grade != "M" & key_alg == "ECC" & {only_private}', columns_to_display=columns_to_display)

# Finalize dataframe
analysis_df = finalize_dataframe(dataframe=analysis_df, column_to_sort=column_to_sort, ascending=sort_ascending, columns_to_display=columns_to_display)
display(analysis_df)

# save to csv
save_table(analysis_df, category=category, table_name=table_name)

,Region,Global #,Without Ssl (Public) #,Without Ssl (Public) %,Without Ssl (Private) #,Without Ssl (Private) %,Rsa (Public) #,Rsa (Public) %,Rsa (Private) #,Rsa (Private) %,Ecc (Public) #,Ecc (Public) %,Ecc (Private) #,Ecc (Private) %
0,California,2,0,0.0,0,0.0,0,0.0,2,100.0,0,0.0,0,0.0
1,Colorado,1,0,0.0,0,0.0,1,100.0,0,0.0,0,0.0,0,0.0
2,Florida,1,0,0.0,0,0.0,0,0.0,1,100.0,0,0.0,0,0.0
3,Georgia,1,0,0.0,0,0.0,0,0.0,1,100.0,0,0.0,0,0.0
4,Michigan,1,0,0.0,0,0.0,0,0.0,1,100.0,0,0.0,0,0.0
5,Minnesota,1,0,0.0,0,0.0,0,0.0,1,100.0,0,0.0,0,0.0
6,New York,1,0,0.0,0,0.0,0,0.0,1,100.0,0,0.0,0,0.0
7,Texas,2,0,0.0,0,0.0,0,0.0,2,100.0,0,0.0,0,0.0
8,Total,10,0,0.0,0,0.0,1,10.0,9,90.0,0,0.0,0,0.0


In [55]:
# Report in latex
report_results = get_extreme_values(analysis_df)
report_name = 'SSL_Algorithms_by_region'

hei_public_rsa = format(report_results.get("Total").get("Rsa (Public) %"), ".2f")
hei_public_ecc = format(report_results.get("Total").get("Ecc (Public) %"), ".2f")

hei_private_rsa = format(report_results.get("Total").get("Rsa (Private) %"), ".2f")
hei_private_ecc = format(report_results.get("Total").get("Ecc (Private) %"), ".2f")

report_figure = f"""
\\begin{{figure}}[htbp]
    \centering
    \includegraphics[width=0.48\\textwidth]{{charts/{report_name}.pdf}}
    \caption{{Distribution of the type of \gls{{ssl}}/\gls{{tls}} algorithms used}}\label{{fig:ssl-algorithms}}
\end{{figure}}
"""

report = f'{report_figure}\n\n'
report += f"""
Fig.~\\ref{{fig:ssl-algorithms}} presents an overview of the use of the type of \gls{{ssl}}/\gls{{tls}} at \glspl{{hei}} in \countryName.

According to the data, {hei_public_rsa}\% of the public institutions analyzed, and {hei_private_rsa}\% of the private institutions are using \gls{{rsa}} encryption algorithm for \gls{{ssl}}/\gls{{tls}}.

On the other hand, {hei_public_ecc}\% of the public institutions analyzed, and {hei_private_ecc}\% of the private institutions are using \gls{{ecc}} encryption algorithm for \gls{{ssl}}/\gls{{tls}}.

In terms of regional differences, private institutions in {report_results.get("Rsa (Private) %").get("top_regions")[0][0]} ({format(report_results.get("Rsa (Private) %").get("top_regions")[0][1], ".2f")}\%), {report_results.get("Rsa (Private) %").get("top_regions")[1][0]} ({format(report_results.get("Rsa (Private) %").get("top_regions")[1][1], ".2f")}\%), and {report_results.get("Rsa (Private) %").get("top_regions")[2][0]} ({format(report_results.get("Rsa (Private) %").get("top_regions")[2][1], ".2f")}\%), and public institutions in {report_results.get("Rsa (Public) %").get("top_regions")[0][0]} ({format(report_results.get("Rsa (Public) %").get("top_regions")[0][1], ".2f")}\%), {report_results.get("Rsa (Public) %").get("top_regions")[1][0]} ({format(report_results.get("Rsa (Public) %").get("top_regions")[1][1], ".2f")}\%), and {report_results.get("Rsa (Public) %").get("top_regions")[2][0]} ({format(report_results.get("Rsa (Public) %").get("top_regions")[2][1], ".2f")}\%) have a higher usage of \gls{{rsa}} encryption algorithm on your websites.

In contrast, private institutions in, {report_results.get("Rsa (Private) %").get("bottom_regions")[0][0]} ({format(report_results.get("Rsa (Private) %").get("bottom_regions")[0][1], ".2f")}\%), {report_results.get("Rsa (Private) %").get("bottom_regions")[1][0]} ({format(report_results.get("Rsa (Private) %").get("bottom_regions")[1][1], ".2f")}\%), and {report_results.get("Rsa (Private) %").get("bottom_regions")[2][0]} ({format(report_results.get("Rsa (Private) %").get("bottom_regions")[2][1], ".2f")}\%), and public institutions in {report_results.get("Rsa (Public) %").get("bottom_regions")[0][0]} ({format(report_results.get("Rsa (Public) %").get("bottom_regions")[0][1], ".2f")}\%), {report_results.get("Rsa (Public) %").get("bottom_regions")[1][0]} ({format(report_results.get("Rsa (Public) %").get("bottom_regions")[1][1], ".2f")}\%), and {report_results.get("Rsa (Public) %").get("bottom_regions")[2][0]} ({format(report_results.get("Rsa (Public) %").get("bottom_regions")[2][1], ".2f")}\%) have a lower usage of \gls{{rsa}} encryption algorithm on your websites.

Finally, private institutions in {report_results.get("Ecc (Private) %").get("top_regions")[0][0]} ({format(report_results.get("Ecc (Private) %").get("top_regions")[0][1], ".2f")}\%), {report_results.get("Ecc (Private) %").get("top_regions")[1][0]} ({format(report_results.get("Ecc (Private) %").get("top_regions")[1][1], ".2f")}\%), and {report_results.get("Ecc (Private) %").get("top_regions")[2][0]} ({format(report_results.get("Ecc (Private) %").get("top_regions")[2][1], ".2f")}\%), and public institutions in {report_results.get("Ecc (Public) %").get("top_regions")[0][0]} ({format(report_results.get("Ecc (Public) %").get("top_regions")[0][1], ".2f")}\%), {report_results.get("Ecc (Public) %").get("top_regions")[1][0]} ({format(report_results.get("Ecc (Public) %").get("top_regions")[1][1], ".2f")}\%), and {report_results.get("Ecc (Public) %").get("top_regions")[2][0]} ({format(report_results.get("Ecc (Public) %").get("top_regions")[2][1], ".2f")}\%) have a higher usage of \gls{{ecc}} encryption algorithm on your websites.

In contrast, private institutions in {report_results.get("Ecc (Private) %").get("bottom_regions")[0][0]} ({format(report_results.get("Ecc (Private) %").get("bottom_regions")[0][1], ".2f")}\%), {report_results.get("Ecc (Private) %").get("bottom_regions")[1][0]} ({format(report_results.get("Ecc (Private) %").get("bottom_regions")[1][1], ".2f")}\%), and {report_results.get("Ecc (Private) %").get("bottom_regions")[2][0]} ({format(report_results.get("Ecc (Private) %").get("bottom_regions")[2][1], ".2f")}\%), and public institutions in {report_results.get("Ecc (Public) %").get("bottom_regions")[0][0]} ({format(report_results.get("Ecc (Public) %").get("bottom_regions")[0][1], ".2f")}\%), {report_results.get("Ecc (Public) %").get("bottom_regions")[1][0]} ({format(report_results.get("Ecc (Public) %").get("bottom_regions")[1][1], ".2f")}\%), and {report_results.get("Ecc (Public) %").get("bottom_regions")[2][0]} ({format(report_results.get("Ecc (Public) %").get("bottom_regions")[2][1], ".2f")}\%) have a lower usage of \gls{{ecc}} encryption algorithm on your websites.
"""

print(report)
# save report to file txt
save_report(report=report, category=category, report_name=report_name)


\begin{figure}[htbp]
    \centering
    \includegraphics[width=0.48\textwidth]{charts/SSL_Algorithms_by_region.pdf}
    \caption{Distribution of the type of \gls{ssl}/\gls{tls} algorithms used}\label{fig:ssl-algorithms}
\end{figure}



Fig.~\ref{fig:ssl-algorithms} presents an overview of the use of the type of \gls{ssl}/\gls{tls} at \glspl{hei} in \countryName.

According to the data, 10.00\% of the public institutions analyzed, and 90.00\% of the private institutions are using \gls{rsa} encryption algorithm for \gls{ssl}/\gls{tls}.

On the other hand, 0.00\% of the public institutions analyzed, and 0.00\% of the private institutions are using \gls{ecc} encryption algorithm for \gls{ssl}/\gls{tls}.

In terms of regional differences, private institutions in California (100.00\%), Florida (100.00\%), and Georgia (100.00\%), and public institutions in Colorado (100.00\%), California (0.00\%), and Florida (0.00\%) have a higher usage of \gls{rsa} encryption algorithm on your websites.

I

In [56]:
# Analyze of Worst supported SSL/TLS versions by region

# settings
column_to_sort = 'Without SSL (Public) %'
sort_ascending = False
table_name = 'Worst_SSL_supported_by_region'

columns_to_display = [region_column_name.title(), column_name_to_results_global]
analysis_df = get_records_by_region(source_df)

# create columns
# Column creation with distribution of records without SSL by region
only_public = 'category == "Public"'
only_private = 'category == "Private"'
create_column(source_df=source_df, analysis_dataframe=analysis_df, column_name='Without SSL (Public)', criteria=f'grade == "M" & {only_public}', columns_to_display=columns_to_display)
create_column(source_df=source_df, analysis_dataframe=analysis_df, column_name='Without SSL (Private)', criteria=f'grade == "M" & {only_private}', columns_to_display=columns_to_display)
# Creating column with the distribution of Worst supported SSL/TLS versions by region
versions = ['SSLv2.0', 'SSLv3.0', 'TLSv1.0', 'TLSv1.1', 'TLSv1.2', 'TLSv1.3']
only_https = 'grade != "M" &'
for i in range(len(versions)):
    current_version = f'`{versions[i]}` == True &' if i != 0 else f'`{versions[i]}` == True'
    previous_versions = ' & '.join([f'`{versions[j]}` == False' for j in range(i)])
    criteria = f'{only_https} {current_version} {previous_versions}'
    create_column(source_df=source_df, analysis_dataframe=analysis_df, column_name=f'{versions[i]} (Public)', criteria=f'{criteria} & {only_public}', columns_to_display=columns_to_display)
    create_column(source_df=source_df, analysis_dataframe=analysis_df, column_name=f'{versions[i]} (Private)', criteria=f'{criteria} & {only_private}', columns_to_display=columns_to_display)

# Finalize dataframe
analysis_df = finalize_dataframe(dataframe=analysis_df, column_to_sort=column_to_sort, ascending=sort_ascending, columns_to_display=columns_to_display)
display(analysis_df)

# save to csv
save_table(analysis_df, category=category, table_name=table_name)

,Region,Global #,Without Ssl (Public) #,Without Ssl (Public) %,Without Ssl (Private) #,Without Ssl (Private) %,Sslv2.0 (Public) #,Sslv2.0 (Public) %,Sslv2.0 (Private) #,Sslv2.0 (Private) %,...,Tlsv1.1 (Private) #,Tlsv1.1 (Private) %,Tlsv1.2 (Public) #,Tlsv1.2 (Public) %,Tlsv1.2 (Private) #,Tlsv1.2 (Private) %,Tlsv1.3 (Public) #,Tlsv1.3 (Public) %,Tlsv1.3 (Private) #,Tlsv1.3 (Private) %
0,California,2,0,0.0,0,0.0,0,0.0,0,0.0,...,0,0.0,0,0.0,2,100.0,0,0.0,0,0.0
1,Colorado,1,0,0.0,0,0.0,0,0.0,0,0.0,...,0,0.0,1,100.0,0,0.0,0,0.0,0,0.0
2,Florida,1,0,0.0,0,0.0,0,0.0,0,0.0,...,0,0.0,0,0.0,1,100.0,0,0.0,0,0.0
3,Georgia,1,0,0.0,0,0.0,0,0.0,0,0.0,...,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0
4,Michigan,1,0,0.0,0,0.0,0,0.0,0,0.0,...,0,0.0,0,0.0,1,100.0,0,0.0,0,0.0
5,Minnesota,1,0,0.0,0,0.0,0,0.0,0,0.0,...,0,0.0,0,0.0,1,100.0,0,0.0,0,0.0
6,New York,1,0,0.0,0,0.0,0,0.0,0,0.0,...,0,0.0,0,0.0,1,100.0,0,0.0,0,0.0
7,Texas,2,0,0.0,0,0.0,0,0.0,0,0.0,...,0,0.0,0,0.0,2,100.0,0,0.0,0,0.0
8,Total,10,0,0.0,0,0.0,0,0.0,0,0.0,...,0,0.0,1,10.0,8,80.0,0,0.0,0,0.0


In [57]:
# Report in latex
report_results = get_extreme_values(analysis_df)
report_name = 'Worst_SSL_supported_by_region'

hei_public_ssl2 = format(report_results.get("Total").get("Sslv2.0 (Public) %"), ".2f")
hei_public_ssl3 = format(report_results.get("Total").get("Sslv3.0 (Public) %"), ".2f")
hei_public_tls10 = format(report_results.get("Total").get("Tlsv1.0 (Public) %"), ".2f")
hei_public_tls11 = format(report_results.get("Total").get("Tlsv1.1 (Public) %"), ".2f")
hei_public_tls12 = format(report_results.get("Total").get("Tlsv1.2 (Public) %"), ".2f")
hei_public_tls13 = format(report_results.get("Total").get("Tlsv1.3 (Public) %"), ".2f")

hei_private_ssl2 = format(report_results.get("Total").get("Sslv2.0 (Private) %"), ".2f")
hei_private_ssl3 = format(report_results.get("Total").get("Sslv3.0 (Private) %"), ".2f")
hei_private_tls10 = format(report_results.get("Total").get("Tlsv1.0 (Private) %"), ".2f")
hei_private_tls11 = format(report_results.get("Total").get("Tlsv1.1 (Private) %"), ".2f")
hei_private_tls12 = format(report_results.get("Total").get("Tlsv1.2 (Private) %"), ".2f")
hei_private_tls13 = format(report_results.get("Total").get("Tlsv1.3 (Private) %"), ".2f")

report_figure = f"""
\\begin{{figure}}[htbp]
    \centering
    \includegraphics[width=0.48\\textwidth]{{charts/{report_name}.pdf}}
    \caption{{Distribution of the worst version of \gls{{ssl}}/\gls{{tls}} protocols.}}\label{{fig:ssl-worst}}
\end{{figure}}
"""

report = f'{report_figure}\n\n'
report += f"""
Fig.~\\ref{{fig:ssl-worst}} presents an overview of the worst version of \gls{{ssl}}/\gls{{tls}} protocols at \glspl{{hei}} in \countryName.

According to the data, {hei_public_ssl2}\% of the public institutions analyzed, and {hei_private_ssl2}\% of the private institutions are using SSLv2.0 version protocol, {hei_public_ssl3}\% of the public institutions analyzed, and {hei_private_ssl3}\% of the private institutions are using SSLv3.0 version protocol, {hei_public_tls10}\% of the public institutions analyzed, and {hei_private_tls10}\% of the private institutions are using TLSv1.0 version protocol, {hei_public_tls11}\% of the public institutions analyzed, and {hei_private_tls11}\% of the private institutions are using TLSv1.1 version protocol, {hei_public_tls12}\% of the public institutions analyzed, and {hei_private_tls12}\% of the private institutions are using TLSv1.2 version protocol, and {hei_public_tls13}\% of the public institutions analyzed, and {hei_private_tls13}\% of the private institutions are using TLSv1.3 version protocol.

In terms of regional differences, private institutions in {report_results.get("Sslv2.0 (Private) %").get("top_regions")[0][0]} ({format(report_results.get("Sslv2.0 (Private) %").get("top_regions")[0][1], ".2f")}\%), {report_results.get("Sslv2.0 (Private) %").get("top_regions")[1][0]} ({format(report_results.get("Sslv2.0 (Private) %").get("top_regions")[1][1], ".2f")}\%), and {report_results.get("Sslv2.0 (Private) %").get("top_regions")[2][0]} ({format(report_results.get("Sslv2.0 (Private) %").get("top_regions")[2][1], ".2f")}\%), and public institutions in {report_results.get("Sslv2.0 (Public) %").get("top_regions")[0][0]} ({format(report_results.get("Sslv2.0 (Public) %").get("top_regions")[0][1], ".2f")}\%), {report_results.get("Sslv2.0 (Public) %").get("top_regions")[1][0]} ({format(report_results.get("Sslv2.0 (Public) %").get("top_regions")[1][1], ".2f")}\%), and {report_results.get("Sslv2.0 (Public) %").get("top_regions")[2][0]} ({format(report_results.get("Sslv2.0 (Public) %").get("top_regions")[2][1], ".2f")}\%) have a higher usage of SSLv2.0 version protocol on your websites.

In contrast, private institutions in {report_results.get("Sslv2.0 (Private) %").get("bottom_regions")[0][0]} ({format(report_results.get("Sslv2.0 (Private) %").get("bottom_regions")[0][1], ".2f")}\%), {report_results.get("Sslv2.0 (Private) %").get("bottom_regions")[1][0]} ({format(report_results.get("Sslv2.0 (Private) %").get("bottom_regions")[1][1], ".2f")}\%), and {report_results.get("Sslv2.0 (Private) %").get("bottom_regions")[2][0]} ({format(report_results.get("Sslv2.0 (Private) %").get("bottom_regions")[2][1], ".2f")}\%), and public institutions in {report_results.get("Sslv2.0 (Public) %").get("bottom_regions")[0][0]} ({format(report_results.get("Sslv2.0 (Public) %").get("bottom_regions")[0][1], ".2f")}\%), {report_results.get("Sslv2.0 (Public) %").get("bottom_regions")[1][0]} ({format(report_results.get("Sslv2.0 (Public) %").get("bottom_regions")[1][1], ".2f")}\%), and {report_results.get("Sslv2.0 (Public) %").get("bottom_regions")[2][0]} ({format(report_results.get("Sslv2.0 (Public) %").get("bottom_regions")[2][1], ".2f")}\%) have a lower usage of SSLv2.0 version protocol on your websites.

While the usage of SSLv3.0 version protocol is higher in private institutions in {report_results.get("Sslv3.0 (Private) %").get("top_regions")[0][0]} ({format(report_results.get("Sslv3.0 (Private) %").get("top_regions")[0][1], ".2f")}\%), {report_results.get("Sslv3.0 (Private) %").get("top_regions")[1][0]} ({format(report_results.get("Sslv3.0 (Private) %").get("top_regions")[1][1], ".2f")}\%), and {report_results.get("Sslv3.0 (Private) %").get("top_regions")[2][0]} ({format(report_results.get("Sslv3.0 (Private) %").get("top_regions")[2][1], ".2f")}\%), and public institutions in {report_results.get("Sslv3.0 (Public) %").get("top_regions")[0][0]} ({format(report_results.get("Sslv3.0 (Public) %").get("top_regions")[0][1], ".2f")}\%), {report_results.get("Sslv3.0 (Public) %").get("top_regions")[1][0]} ({format(report_results.get("Sslv3.0 (Public) %").get("top_regions")[1][1], ".2f")}\%), and {report_results.get("Sslv3.0 (Public) %").get("top_regions")[2][0]} ({format(report_results.get("Sslv3.0 (Public) %").get("top_regions")[2][1], ".2f")}\%) than in other regions, the usage of SSLv3.0 version protocol is lower in private institutions in {report_results.get("Sslv3.0 (Private) %").get("bottom_regions")[0][0]} ({format(report_results.get("Sslv3.0 (Private) %").get("bottom_regions")[0][1], ".2f")}\%), {report_results.get("Sslv3.0 (Private) %").get("bottom_regions")[1][0]} ({format(report_results.get("Sslv3.0 (Private) %").get("bottom_regions")[1][1], ".2f")}\%), and {report_results.get("Sslv3.0 (Private) %").get("bottom_regions")[2][0]} ({format(report_results.get("Sslv3.0 (Private) %").get("bottom_regions")[2][1], ".2f")}\%), and public institutions in {report_results.get("Sslv3.0 (Public) %").get("bottom_regions")[0][0]} ({format(report_results.get("Sslv3.0 (Public) %").get("bottom_regions")[0][1], ".2f")}\%), {report_results.get("Sslv3.0 (Public) %").get("bottom_regions")[1][0]} ({format(report_results.get("Sslv3.0 (Public) %").get("bottom_regions")[1][1], ".2f")}\%), and {report_results.get("Sslv3.0 (Public) %").get("bottom_regions")[2][0]} ({format(report_results.get("Sslv3.0 (Public) %").get("bottom_regions")[2][1], ".2f")}\%) than in other regions.

The usage of TLSv1.0 version protocol is higher in private institutions in {report_results.get("Tlsv1.0 (Private) %").get("top_regions")[0][0]} ({format(report_results.get("Tlsv1.0 (Private) %").get("top_regions")[0][1], ".2f")}\%), {report_results.get("Tlsv1.0 (Private) %").get("top_regions")[1][0]} ({format(report_results.get("Tlsv1.0 (Private) %").get("top_regions")[1][1], ".2f")}\%), and {report_results.get("Tlsv1.0 (Private) %").get("top_regions")[2][0]} ({format(report_results.get("Tlsv1.0 (Private) %").get("top_regions")[2][1], ".2f")}\%), and public institutions in {report_results.get("Tlsv1.0 (Public) %").get("top_regions")[0][0]} ({format(report_results.get("Tlsv1.0 (Public) %").get("top_regions")[0][1], ".2f")}\%), {report_results.get("Tlsv1.0 (Public) %").get("top_regions")[1][0]} ({format(report_results.get("Tlsv1.0 (Public) %").get("top_regions")[1][1], ".2f")}\%), and {report_results.get("Tlsv1.0 (Public) %").get("top_regions")[2][0]} ({format(report_results.get("Tlsv1.0 (Public) %").get("top_regions")[2][1], ".2f")}\%) than in other regions, while the usage of TLSv1.0 version protocol is lower in private institutions in {report_results.get("Tlsv1.0 (Private) %").get("bottom_regions")[0][0]} ({format(report_results.get("Tlsv1.0 (Private) %").get("bottom_regions")[0][1], ".2f")}\%), {report_results.get("Tlsv1.0 (Private) %").get("bottom_regions")[1][0]} ({format(report_results.get("Tlsv1.0 (Private) %").get("bottom_regions")[1][1], ".2f")}\%), and {report_results.get("Tlsv1.0 (Private) %").get("bottom_regions")[2][0]} ({format(report_results.get("Tlsv1.0 (Private) %").get("bottom_regions")[2][1], ".2f")}\%), and public institutions in {report_results.get("Tlsv1.0 (Public) %").get("bottom_regions")[0][0]} ({format(report_results.get("Tlsv1.0 (Public) %").get("bottom_regions")[0][1], ".2f")}\%), {report_results.get("Tlsv1.0 (Public) %").get("bottom_regions")[1][0]} ({format(report_results.get("Tlsv1.0 (Public) %").get("bottom_regions")[1][1], ".2f")}\%), and {report_results.get("Tlsv1.0 (Public) %").get("bottom_regions")[2][0]} ({format(report_results.get("Tlsv1.0 (Public) %").get("bottom_regions")[2][1], ".2f")}\%) than in other regions.

Already the use of TLSv1.1 version protocol is higher in private institutions in {report_results.get("Tlsv1.1 (Private) %").get("top_regions")[0][0]} ({format(report_results.get("Tlsv1.1 (Private) %").get("top_regions")[0][1], ".2f")}\%), {report_results.get("Tlsv1.1 (Private) %").get("top_regions")[1][0]} ({format(report_results.get("Tlsv1.1 (Private) %").get("top_regions")[1][1], ".2f")}\%), and {report_results.get("Tlsv1.1 (Private) %").get("top_regions")[2][0]} ({format(report_results.get("Tlsv1.1 (Private) %").get("top_regions")[2][1], ".2f")}\%), and public institutions in {report_results.get("Tlsv1.1 (Public) %").get("top_regions")[0][0]} ({format(report_results.get("Tlsv1.1 (Public) %").get("top_regions")[0][1], ".2f")}\%), {report_results.get("Tlsv1.1 (Public) %").get("top_regions")[1][0]} ({format(report_results.get("Tlsv1.1 (Public) %").get("top_regions")[1][1], ".2f")}\%), and {report_results.get("Tlsv1.1 (Public) %").get("top_regions")[2][0]} ({format(report_results.get("Tlsv1.1 (Public) %").get("top_regions")[2][1], ".2f")}\%) than in other regions, while the usage of TLSv1.1 version protocol is lower in private institutions in {report_results.get("Tlsv1.1 (Private) %").get("bottom_regions")[0][0]} ({format(report_results.get("Tlsv1.1 (Private) %").get("bottom_regions")[0][1], ".2f")}\%), {report_results.get("Tlsv1.1 (Private) %").get("bottom_regions")[1][0]} ({format(report_results.get("Tlsv1.1 (Private) %").get("bottom_regions")[1][1], ".2f")}\%), and {report_results.get("Tlsv1.1 (Private) %").get("bottom_regions")[2][0]} ({format(report_results.get("Tlsv1.1 (Private) %").get("bottom_regions")[2][1], ".2f")}\%), and public institutions in {report_results.get("Tlsv1.1 (Public) %").get("bottom_regions")[0][0]} ({format(report_results.get("Tlsv1.1 (Public) %").get("bottom_regions")[0][1], ".2f")}\%), {report_results.get("Tlsv1.1 (Public) %").get("bottom_regions")[1][0]} ({format(report_results.get("Tlsv1.1 (Public) %").get("bottom_regions")[1][1], ".2f")}\%), and {report_results.get("Tlsv1.1 (Public) %").get("bottom_regions")[2][0]} ({format(report_results.get("Tlsv1.1 (Public) %").get("bottom_regions")[2][1], ".2f")}\%) than in other regions.

While the use of TLSv1.2 version protocol is higher in private institutions in {report_results.get("Tlsv1.2 (Private) %").get("top_regions")[0][0]} ({format(report_results.get("Tlsv1.2 (Private) %").get("top_regions")[0][1], ".2f")}\%), {report_results.get("Tlsv1.2 (Private) %").get("top_regions")[1][0]} ({format(report_results.get("Tlsv1.2 (Private) %").get("top_regions")[1][1], ".2f")}\%), and {report_results.get("Tlsv1.2 (Private) %").get("top_regions")[2][0]} ({format(report_results.get("Tlsv1.2 (Private) %").get("top_regions")[2][1], ".2f")}\%), and public institutions in {report_results.get("Tlsv1.2 (Public) %").get("top_regions")[0][0]} ({format(report_results.get("Tlsv1.2 (Public) %").get("top_regions")[0][1], ".2f")}\%), {report_results.get("Tlsv1.2 (Public) %").get("top_regions")[1][0]} ({format(report_results.get("Tlsv1.2 (Public) %").get("top_regions")[1][1], ".2f")}\%), and {report_results.get("Tlsv1.2 (Public) %").get("top_regions")[2][0]} ({format(report_results.get("Tlsv1.2 (Public) %").get("top_regions")[2][1], ".2f")}\%) than in other regions, while the usage of TLSv1.2 version protocol is lower in private institutions in {report_results.get("Tlsv1.2 (Private) %").get("bottom_regions")[0][0]} ({format(report_results.get("Tlsv1.2 (Private) %").get("bottom_regions")[0][1], ".2f")}\%), {report_results.get("Tlsv1.2 (Private) %").get("bottom_regions")[1][0]} ({format(report_results.get("Tlsv1.2 (Private) %").get("bottom_regions")[1][1], ".2f")}\%), and {report_results.get("Tlsv1.2 (Private) %").get("bottom_regions")[2][0]} ({format(report_results.get("Tlsv1.2 (Private) %").get("bottom_regions")[2][1], ".2f")}\%), and public institutions in {report_results.get("Tlsv1.2 (Public) %").get("bottom_regions")[0][0]} ({format(report_results.get("Tlsv1.2 (Public) %").get("bottom_regions")[0][1], ".2f")}\%), {report_results.get("Tlsv1.2 (Public) %").get("bottom_regions")[1][0]} ({format(report_results.get("Tlsv1.2 (Public) %").get("bottom_regions")[1][1], ".2f")}\%), and {report_results.get("Tlsv1.2 (Public) %").get("bottom_regions")[2][0]} ({format(report_results.get("Tlsv1.2 (Public) %").get("bottom_regions")[2][1], ".2f")}\%) than in other regions.

Finally, the use of TLSv1.3 version protocol is higher in private institutions in {report_results.get("Tlsv1.3 (Private) %").get("top_regions")[0][0]} ({format(report_results.get("Tlsv1.3 (Private) %").get("top_regions")[0][1], ".2f")}\%), {report_results.get("Tlsv1.3 (Private) %").get("top_regions")[1][0]} ({format(report_results.get("Tlsv1.3 (Private) %").get("top_regions")[1][1], ".2f")}\%), and {report_results.get("Tlsv1.3 (Private) %").get("top_regions")[2][0]} ({format(report_results.get("Tlsv1.3 (Private) %").get("top_regions")[2][1], ".2f")}\%), and public institutions in {report_results.get("Tlsv1.3 (Public) %").get("top_regions")[0][0]} ({format(report_results.get("Tlsv1.3 (Public) %").get("top_regions")[0][1], ".2f")}\%), {report_results.get("Tlsv1.3 (Public) %").get("top_regions")[1][0]} ({format(report_results.get("Tlsv1.3 (Public) %").get("top_regions")[1][1], ".2f")}\%), and {report_results.get("Tlsv1.3 (Public) %").get("top_regions")[2][0]} ({format(report_results.get("Tlsv1.3 (Public) %").get("top_regions")[2][1], ".2f")}\%) than in other regions, while the usage of TLSv1.3 version protocol is lower in private institutions in {report_results.get("Tlsv1.3 (Private) %").get("bottom_regions")[0][0]} ({format(report_results.get("Tlsv1.3 (Private) %").get("bottom_regions")[0][1], ".2f")}\%), {report_results.get("Tlsv1.3 (Private) %").get("bottom_regions")[1][0]} ({format(report_results.get("Tlsv1.3 (Private) %").get("bottom_regions")[1][1], ".2f")}\%), and {report_results.get("Tlsv1.3 (Private) %").get("bottom_regions")[2][0]} ({format(report_results.get("Tlsv1.3 (Private) %").get("bottom_regions")[2][1], ".2f")}\%), and public institutions in {report_results.get("Tlsv1.3 (Public) %").get("bottom_regions")[0][0]} ({format(report_results.get("Tlsv1.3 (Public) %").get("bottom_regions")[0][1], ".2f")}\%), {report_results.get("Tlsv1.3 (Public) %").get("bottom_regions")[1][0]} ({format(report_results.get("Tlsv1.3 (Public) %").get("bottom_regions")[1][1], ".2f")}\%), and {report_results.get("Tlsv1.3 (Public) %").get("bottom_regions")[2][0]} ({format(report_results.get("Tlsv1.3 (Public) %").get("bottom_regions")[2][1], ".2f")}\%) than in other regions.
"""

print(report)
# save report to file txt
save_report(report=report, category=category, report_name=report_name)



\begin{figure}[htbp]
    \centering
    \includegraphics[width=0.48\textwidth]{charts/Worst_SSL_supported_by_region.pdf}
    \caption{Distribution of the worst version of \gls{ssl}/\gls{tls} protocols.}\label{fig:ssl-worst}
\end{figure}



Fig.~\ref{fig:ssl-worst} presents an overview of the worst version of \gls{ssl}/\gls{tls} protocols at \glspl{hei} in \countryName.

According to the data, 0.00\% of the public institutions analyzed, and 0.00\% of the private institutions are using SSLv2.0 version protocol, 0.00\% of the public institutions analyzed, and 0.00\% of the private institutions are using SSLv3.0 version protocol, 0.00\% of the public institutions analyzed, and 10.00\% of the private institutions are using TLSv1.0 version protocol, 0.00\% of the public institutions analyzed, and 0.00\% of the private institutions are using TLSv1.1 version protocol, 10.00\% of the public institutions analyzed, and 80.00\% of the private institutions are using TLSv1.2 version protocol, and 0

In [58]:
# Analyze of valid SSL/TLS by region

# settings
column_to_sort = 'Without SSL (Public) %'
sort_ascending = False
table_name = 'valid_ssl_by_region'

columns_to_display = [region_column_name.title(), column_name_to_results_global]
analysis_df = get_records_by_region(source_df)

# create columns
# Column creation with distribution of records without SSL by region
only_public = 'category == "Public"'
only_private = 'category == "Private"'
create_column(source_df=source_df, analysis_dataframe=analysis_df, column_name='Without SSL (Public)', criteria=f'grade == "M" & {only_public}', columns_to_display=columns_to_display)
create_column(source_df=source_df, analysis_dataframe=analysis_df, column_name='Without SSL (Private)', criteria=f'grade == "M" & {only_private}', columns_to_display=columns_to_display)
# Creating column with the distribution of valid SSL/TLS by region
create_column(source_df=source_df, analysis_dataframe=analysis_df, column_name='Valid Configuration (Public)', criteria=f'grade != "M" & is_valid == True & {only_public}', columns_to_display=columns_to_display)
create_column(source_df=source_df, analysis_dataframe=analysis_df, column_name='Invalid Configuration (Public)', criteria=f'grade != "M" & is_valid == False & {only_public}', columns_to_display=columns_to_display)
create_column(source_df=source_df, analysis_dataframe=analysis_df, column_name='Valid Configuration (Private)', criteria=f'grade != "M" & is_valid == True & {only_private}', columns_to_display=columns_to_display)
create_column(source_df=source_df, analysis_dataframe=analysis_df, column_name='Invalid Configuration (Private)', criteria=f'grade != "M" & is_valid == False & {only_private}', columns_to_display=columns_to_display)

# Finalize dataframe
analysis_df = finalize_dataframe(dataframe=analysis_df, column_to_sort=column_to_sort, ascending=sort_ascending, columns_to_display=columns_to_display)
display(analysis_df)

# save to csv
save_table(analysis_df, category=category, table_name=table_name)

,Region,Global #,Without Ssl (Public) #,Without Ssl (Public) %,Without Ssl (Private) #,Without Ssl (Private) %,Valid Configuration (Public) #,Valid Configuration (Public) %,Invalid Configuration (Public) #,Invalid Configuration (Public) %,Valid Configuration (Private) #,Valid Configuration (Private) %,Invalid Configuration (Private) #,Invalid Configuration (Private) %
0,California,2,0,0.0,0,0.0,0,0.0,0,0.0,2,100.0,0,0.0
1,Colorado,1,0,0.0,0,0.0,1,100.0,0,0.0,0,0.0,0,0.0
2,Florida,1,0,0.0,0,0.0,0,0.0,0,0.0,1,100.0,0,0.0
3,Georgia,1,0,0.0,0,0.0,0,0.0,0,0.0,1,100.0,0,0.0
4,Michigan,1,0,0.0,0,0.0,0,0.0,0,0.0,1,100.0,0,0.0
5,Minnesota,1,0,0.0,0,0.0,0,0.0,0,0.0,1,100.0,0,0.0
6,New York,1,0,0.0,0,0.0,0,0.0,0,0.0,1,100.0,0,0.0
7,Texas,2,0,0.0,0,0.0,0,0.0,0,0.0,2,100.0,0,0.0
8,Total,10,0,0.0,0,0.0,1,10.0,0,0.0,9,90.0,0,0.0


In [59]:
# Report in latex
report_results = get_extreme_values(analysis_df)
report_name = 'valid_ssl_by_region'

hei_public_valid = format(report_results.get("Total").get("Valid Configuration (Public) %"), ".2f")
hei_public_invalid = format(report_results.get("Total").get("Invalid Configuration (Public) %"), ".2f")

hei_private_valid = format(report_results.get("Total").get("Valid Configuration (Private) %"), ".2f")
hei_private_invalid = format(report_results.get("Total").get("Invalid Configuration (Private) %"), ".2f")

report_figure = f"""
\\begin{{figure}}[htbp]
    \centering
    \includegraphics[width=0.48\\textwidth]{{charts/{report_name}.pdf}}
    \caption{{Distribution of valid configuration \gls{{ssl}}/\gls{{tls}} by region.}}\label{{fig:valid-ssl}}
\end{{figure}}
"""

report = f'{report_figure}\n\n'

report += f"""
Fig.~\\ref{{fig:valid-ssl}} presents an overview of the valid configuration of \gls{{ssl}}/\gls{{tls}} protocols at \glspl{{hei}} in \countryName.

According to the data, \gls{{ssl}}/\gls{{tls}} protocols are configured correctly in {hei_public_valid}\% of the \glspl{{hei}} public that are publicly accessible, and in {hei_private_valid}\% of the \glspl{{hei}} private.

On the other hand, {hei_public_invalid}\% of the \glspl{{hei}} public that are publicly accessible have an invalid configuration of \gls{{ssl}}/\gls{{tls}} protocols, and {hei_private_invalid}\% of the \glspl{{hei}} private.

In terms of regional differences, private institutions in {report_results.get("Valid Configuration (Private) %").get("top_regions")[0][0]} ({format(report_results.get("Valid Configuration (Private) %").get("top_regions")[0][1], ".2f")}\%), {report_results.get("Valid Configuration (Private) %").get("top_regions")[1][0]} ({format(report_results.get("Valid Configuration (Private) %").get("top_regions")[1][1], ".2f")}\%), and {report_results.get("Valid Configuration (Private) %").get("top_regions")[2][0]} ({format(report_results.get("Valid Configuration (Private) %").get("top_regions")[2][1], ".2f")}\%), and public institutions in {report_results.get("Valid Configuration (Public) %").get("top_regions")[0][0]} ({format(report_results.get("Valid Configuration (Public) %").get("top_regions")[0][1], ".2f")}\%), {report_results.get("Valid Configuration (Public) %").get("top_regions")[1][0]} ({format(report_results.get("Valid Configuration (Public) %").get("top_regions")[1][1], ".2f")}\%), and {report_results.get("Valid Configuration (Public) %").get("top_regions")[2][0]} ({format(report_results.get("Valid Configuration (Public) %").get("top_regions")[2][1], ".2f")}\%) have the highest percentage of valid configuration of \gls{{ssl}}/\gls{{tls}} protocols.

In contrast, private institutions in {report_results.get("Invalid Configuration (Private) %").get("top_regions")[0][0]} ({format(report_results.get("Invalid Configuration (Private) %").get("top_regions")[0][1], ".2f")}\%), {report_results.get("Invalid Configuration (Private) %").get("top_regions")[1][0]} ({format(report_results.get("Invalid Configuration (Private) %").get("top_regions")[1][1], ".2f")}\%), and {report_results.get("Invalid Configuration (Private) %").get("top_regions")[2][0]} ({format(report_results.get("Invalid Configuration (Private) %").get("top_regions")[2][1], ".2f")}\%), and public institutions in {report_results.get("Invalid Configuration (Public) %").get("top_regions")[0][0]} ({format(report_results.get("Invalid Configuration (Public) %").get("top_regions")[0][1], ".2f")}\%), {report_results.get("Invalid Configuration (Public) %").get("top_regions")[1][0]} ({format(report_results.get("Invalid Configuration (Public) %").get("top_regions")[1][1], ".2f")}\%), and {report_results.get("Invalid Configuration (Public) %").get("top_regions")[2][0]} ({format(report_results.get("Invalid Configuration (Public) %").get("top_regions")[2][1], ".2f")}\%) have the highest percentage of invalid configuration of \gls{{ssl}}/\gls{{tls}} protocols.
"""

print(report)
# save report to file txt
save_report(report=report, category=category, report_name=report_name)


\begin{figure}[htbp]
    \centering
    \includegraphics[width=0.48\textwidth]{charts/valid_ssl_by_region.pdf}
    \caption{Distribution of valid configuration \gls{ssl}/\gls{tls} by region.}\label{fig:valid-ssl}
\end{figure}



Fig.~\ref{fig:valid-ssl} presents an overview of the valid configuration of \gls{ssl}/\gls{tls} protocols at \glspl{hei} in \countryName.

According to the data, \gls{ssl}/\gls{tls} protocols are configured correctly in 10.00\% of the \glspl{hei} public that are publicly accessible, and in 90.00\% of the \glspl{hei} private.

On the other hand, 0.00\% of the \glspl{hei} public that are publicly accessible have an invalid configuration of \gls{ssl}/\gls{tls} protocols, and 0.00\% of the \glspl{hei} private.

In terms of regional differences, private institutions in California (100.00\%), Florida (100.00\%), and Georgia (100.00\%), and public institutions in Colorado (100.00\%), California (0.00\%), and Florida (0.00\%) have the highest percentage of valid confi